# Time Patterns for Crisis-Related Pages

This notebook shows when people are most likely to view crisis-related pages.

**Focus:** hour-of-day and day-of-week demand patterns
**Data source:** GA4 BigQuery export (`events_*`)


In [ ]:
#@title Setup (run once)
import sys
import os

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    if not os.path.exists("lla-data"):
        !git clone -q https://github.com/aidoanto/lla-data.git
    repo = os.path.abspath("lla-data")
    if repo not in sys.path:
        sys.path.insert(0, repo)
    !pip install -q db-dtypes google-cloud-bigquery kaleido plotly
else:
    for p in ("..", "../.."):
        ap = os.path.abspath(p)
        if ap not in sys.path:
            sys.path.insert(0, ap)

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import lifeline_theme
from lla_data import config
from lla_data.bq import get_client, run_query

lifeline_theme.inject_fonts()

client = get_client()

In [ ]:
#@title Parameters
DAYS_BACK = config.DEFAULT_DAYS_BACK #@param {type:"integer"}

In [ ]:
query = f"""
WITH page_views AS (
  SELECT
    PARSE_DATE('%Y%m%d', event_date) AS event_day,
    EXTRACT(HOUR FROM TIMESTAMP_MICROS(event_timestamp)) AS hour_of_day,
    EXTRACT(DAYOFWEEK FROM PARSE_DATE('%Y%m%d', event_date)) AS day_of_week_num,
    FORMAT_DATE('%A', PARSE_DATE('%Y%m%d', event_date)) AS day_of_week,
    CASE
      WHEN page_location = '(unknown)' THEN '(unknown)'
      WHEN REGEXP_CONTAINS(page_location, r'^https?://') THEN COALESCE(NULLIF(REGEXP_EXTRACT(page_location, r'^https?://[^/]+(/.*)$'), ''), '/')
      WHEN STARTS_WITH(page_location, '/') THEN page_location
      ELSE CONCAT('/', page_location)
    END AS page_path
  FROM (
    SELECT
      event_date,
      event_timestamp,
      COALESCE((
        SELECT ep.value.string_value
        FROM UNNEST(event_params) ep
        WHERE ep.key = 'page_location'
      ), '(unknown)') AS page_location
    FROM `{config.PROJECT_ID}.{config.GA4_DATASET}.events_*`
    WHERE event_name = 'page_view'
      AND _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL {DAYS_BACK} DAY))
      AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
  )
), cleaned AS (
  SELECT
    event_day,
    hour_of_day,
    day_of_week_num,
    day_of_week,
    CASE
      WHEN page_path IN ('(unknown)', '/') THEN page_path
      ELSE REGEXP_REPLACE(REGEXP_REPLACE(page_path, r'#.*$', ''), r'\\?.*$', '')
    END AS page_path_clean
  FROM page_views
), crisis AS (
  SELECT *
  FROM cleaned
  WHERE REGEXP_CONTAINS(page_path_clean, r'^/(get-help|crisis-support|suicide|131114|chat|text)')
)
SELECT
  day_of_week_num,
  day_of_week,
  hour_of_day,
  COUNT(*) AS crisis_page_views
FROM crisis
GROUP BY day_of_week_num, day_of_week, hour_of_day
ORDER BY day_of_week_num, hour_of_day
"""

df = run_query(client, query)
df.head()

In [ ]:
day_order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
df["day_of_week"] = pd.Categorical(df["day_of_week"], categories=day_order, ordered=True)

heatmap_df = (
    df.pivot_table(
        index="day_of_week",
        columns="hour_of_day",
        values="crisis_page_views",
        aggfunc="sum",
        fill_value=0,
    )
    .sort_index()
)

fig = px.imshow(
    heatmap_df,
    labels={"x": "Hour of Day", "y": "Day of Week", "color": "Crisis Page Views"},
    color_continuous_scale="YlOrRd",
    aspect="auto",
    template="lifeline",
    title=f"Crisis-Related Page Views by Time (Last {DAYS_BACK} Days)",
)
fig.update_xaxes(dtick=1)
lifeline_theme.add_lifeline_logo(fig)
fig.show()

In [ ]:
hour_summary = (
    df.groupby("hour_of_day", as_index=False)["crisis_page_views"]
    .sum()
    .sort_values("hour_of_day")
)

fig = go.Figure(
    data=[
        go.Scatter(
            x=hour_summary["hour_of_day"],
            y=hour_summary["crisis_page_views"],
            mode="lines+markers",
            name="Crisis Page Views",
        )
    ]
)
fig.update_layout(
    template="lifeline",
    title=f"Hourly Demand Curve for Crisis-Related Content (Last {DAYS_BACK} Days)",
    xaxis_title="Hour of Day",
    yaxis_title="Page Views",
)
lifeline_theme.add_lifeline_logo(fig)
fig.show()